In [5]:
!pip install python-dotenv openai pandas

In [32]:
from typing import Dict, List
from utilities import (
    num_tokens_from_messages,
    get_embedding,
    get_n_nearest_neighbors,
    memoize_to_sqlite,
)
from f1_utilities import wikipedia_splitter, Section
import requests
import os
import itertools
import tiktoken
import openai
import pandas as pd

TypeError: Too many arguments for typing.List; actual 2, expected 1

SyntaxError: from __future__ imports must occur at the beginning of the file (__init__.py, line 3)